In [1]:
import serial
import time
import pandas as pd
import numpy as np
import serial.tools.list_ports 

# CONFIGURATION
SERIAL_PORT = 'COM6'  # CHECK YOUR PORT
BAUD_RATE = 115200

# 1. LOAD DATA
print("Loading Test Data...")
X_test = pd.read_csv('x_testing.csv') 
# Load Y, ignore header, and flatten to simple list
y_df = pd.read_csv('y_testing.csv', header=None)
y_test = y_df.values.flatten()
def run_demo(X_data, y_data):
    True_predicted=0
    total=len(y_data)
    
    print(f"Attempting to connect to {SERIAL_PORT}...")
    
    try:
        # TIMEOUT increased to 2 seconds to be safe
        arduino = serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=2) 
        time.sleep(2) # Wait for Arduino reboot
        print(f"SUCCESS: Connected to Fog Node on {SERIAL_PORT}")
        print("Clearing startup messages...")
        arduino.reset_input_buffer() 
        while arduino.in_waiting > 0:
            line = arduino.readline().decode('utf-8').strip()
            print(f"   [Ignored Startup Log]: {line}")
        print("Buffer is clean. Starting Stream.\n")
    except Exception as e:
        print(f"Error connecting: {e}")
        return

    print("\n--- STARTING TRAFFIC STREAM ---")
    
    try: 
        # Loop through first 10 packets
        for i in range(0, 10):
            
            features = X_data.iloc[i].values
            true_label = y_data[i]

            # Format data
            msg = ",".join(["{:.4f}".format(x) for x in features]) + "\n"
            
            print(f"Sending Packet #{i} (True: {true_label})...")
            
            # 1. Send
            arduino.write(bytes(msg, 'utf-8'))
            
            # 2. Read Response (Blocking)
            # Since Arduino sends reply BEFORE the delay now, we get this instantly
            response = arduino.readline().decode('utf-8').strip()
            
            if response:
                print(f"--> Fog Response: {response}")
            else:
                print("--> No response (Timeout).")
            print("********")
            result=response.split('-')
            # print(result[0])
            # break
            if((result[0] =="Intrusion" and (true_label== 1 or true_label ==2)) or(result[0] =="BENIGN" and (true_label== 0))):
                True_predicted+=1
            else:
                total-=1
                
                
            print("-" * 30)

            # 3. SYNC DELAY
            # Arduino is sleeping for 2.0 seconds.
            # We sleep for 2.5 seconds to make sure Arduino is awake 
            # before we send the next packet.
            time.sleep(2.5) 

    except Exception as e:
        print(f"Error: {e}")
        
    finally:
        arduino.close()
        print("Connection Closed.")
    
    return total,True_predicted

# Run
total_remain,True_pred =run_demo(X_test, y_test)
print("*************************************")
print("\nTrue Predicted:",True_pred)
print("\nWrong predicted:",len(y_test)-total_remain)


Loading Test Data...
Attempting to connect to COM6...
SUCCESS: Connected to Fog Node on COM6
Clearing startup messages...
Buffer is clean. Starting Stream.


--- STARTING TRAFFIC STREAM ---
Sending Packet #0 (True: 0)...
--> Fog Response: BENIGN-| Class: 0 | Latency: 216 us
********
------------------------------
Sending Packet #1 (True: 0)...
--> Fog Response: BENIGN-| Class: 0 | Latency: 272 us
********
------------------------------
Sending Packet #2 (True: 0)...
--> Fog Response: BENIGN-| Class: 0 | Latency: 224 us
********
------------------------------
Sending Packet #3 (True: 1)...
--> Fog Response: Intrusion-| Class: 1 | Latency: 280 us
********
------------------------------
Sending Packet #4 (True: 1)...
--> Fog Response: Intrusion-| Class: 1 | Latency: 272 us
********
------------------------------
Sending Packet #5 (True: 0)...
--> Fog Response: BENIGN-| Class: 0 | Latency: 228 us
********
------------------------------
Sending Packet #6 (True: 0)...
--> Fog Response: BENIG

In [54]:
import pandas as pd

# Read the CSV files
x_df = pd.read_csv('x_testing.csv')
y_df = pd.read_csv('y_testing.csv')

# Drop the first column from both
# The snippet suggests the first column might be an unnamed index column.
# I'll drop the column at index 0.
x_df = x_df.iloc[:, 1:]
y_df = y_df.iloc[:, 1:]

# Save the modified dataframes to new CSV files
x_df.to_csv('x_testing_cleaned.csv', index=False)
y_df.to_csv('y_testing_cleaned.csv', index=False)

print("Columns removed.")
print("x_testing_cleaned.csv shape:", x_df.shape)
print("y_testing_cleaned.csv shape:", y_df.shape)

Columns removed.
x_testing_cleaned.csv shape: (300, 38)
y_testing_cleaned.csv shape: (300, 1)
